In [1]:
import numpy as np

### 1. Определим класс узлов бинарного дерева

In [2]:
class BinTreeNode:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right
        
    @property
    def value(self):
        '''
        value getter: возвращает значение узла
        '''
        
        return self.__value
    
    @value.setter
    def value(self, value):
        '''
        value setter: задает значение узла
        '''
        
        self.__value = value
        
    @property
    def left(self):
        '''
        getter left: возвращает ссылку на левое поддерево
        '''
        return self.__left
    
    @left.setter
    def left(self, left):
        '''
        setter left: задает ссылку на левое поддерево
        '''
        self.__left = left
    
    @property
    def right(self):
        '''
        getter right: возвращает ссылку на правое поддерово
        '''
        return self.__right
    
    @right.setter
    def right(self, right):
        '''
        getter right: задет ссылку на правое поддерово
        '''
        self.__right = right

### 2. Определим класс бинарного дерева поиска

In [3]:
class BinTree:
    '''
    Реализация бинарного дерева поиска. Вставляемые элементы должны быть сравнимы.
    '''
    
    def __init__(self):
        self.root = None
        
    def __is_left_child(self, node, parent):
        '''
        Проверяет, является ли узел правым потомком.
        '''
        if node.value < parent.value:
            return True
        
        return False
    
    def __min(self, node, parent):
        '''
        Рекурсивный поиск минимального элемента.
        '''
        if node.left is None:
            return node, parent
            
        return __min(node.left, node)
    
    def __max(self, node, parent):
        '''
        Рекурсивный поиск максимального элемента.
        '''
        if node.right is None:
            return node, parent
        
        return __max(node.right, node)
    
    def __iterate_tree(self, root):
        '''
        Генератор упорядоченных значений элементов дерева.
        '''
        if root == None:
            return
        
        yield from self.__iterate_tree(root.left)
        yield root.value
        yield from self.__iterate_tree(root.right)
    
    def __search_value(self, root, parent, value):
        '''
        Рекурсивный поиск значения в дереве.
        Возвращет узел с даным значением и его родителя
        '''
        
        if root == None:
            return None, parent
        
        if value == root.value:
            return root, parent
        
        if value < root.value:
            return self.__search_value(root.left, root, value)
        
        return self.__search_value(root.right, root, value)
    
    def __iter__(self):
        '''
        Конвертирует объект в итератор.
        '''
        return self.__iterate_tree(self.root)
    
    def __len__(self):
        '''
        Возвращает количество элементов в дереве.
        '''
        return sum(1 for _ in iter(self))
    
    def __add__(self, value):
        '''
        Перегруженная операция '+' для вставки элемента в дерево.
        '''
        self.insert(value)
        
    def __sub__(self, value):
        '''
        Перегруженная операция '-' для удаления элемента из дерева.
        '''
        self.delete(value)
        
    def max_item(self):
        '''
        Возвращает максимальный элемент дерева.
        '''
        
        if self.root is None:
            return None
        
        node, _ = __max(self.root)
        return node
    
    def min_item(self):
        '''
        Возвращает минимальный элемент дерева.
        '''
        
        if self.root is None:
            return None
        node, _ = __min(self.root)
        return node
    
    def insert(self, value):
        '''
        Вставка элемента в дерево.
        Использует рекурсивную функцию __search_value для поиска родителя
        вставляемой вершины.
        '''
        
        node = BinTreeNode(value)
        
        if self.root is None:
            self.root = node
            return
        
        exist_node, parent = self.__search_value(self.root, None, value)
        if not (exist_node is None):
            return
        
        if value < parent.value:
            parent.left = node
        else:
            parent.right = node
    
    def delete(self, value):
        '''
        Удаление элемента из дерева.
        Использует рекурсивную функцию __search_value для поиска родителя
        удаляемой вершины.
        '''
        
        if self.root is None:
            return False
        
        node, parent = self.__search_value(self.root, None, value)
        
        if node is None:
            return False
        
        # Узел - лист
        if node.left is None and node.right is None:
            if parent is None:
                self.root = None
                return True
            
            if self.__is_left_child(node, parent):
                parent.left = None
            else:
                parent.right = None
            
            return True
        
        # Узел с одним дочерним узлом
        if not (node.left is None) and node.right is None:
            if parent is None:
                self.root = node.left
                return True
            
            parent.left = node.left
            return True
        elif node.left is None and not (node.right is None):
            if parent is None:
                self.root = node.right
                return True
            
            parent.right = node.right
            return True
        
        # Узел с двумя дочерними узлами
        min_node, min_parent = self.__min(node.right, node)
        if min_parent != node:
            min_parent.left = None
        
        if parent is None:
            self.root = min_node
        elif self.__is_left_child(node, parent):
            parent.left = min_node
        else:
            parent.right = min_node
        
        min_node.left = node.left
            
        return True
            
    
    def search(self, value):
        '''
        Поиск узла с заданным значением.
        '''
        
        node, _ = self.__search_value(self.root, None, value)
        return node
    
    def contains(self, value):
        '''
        Проверка наличия элемента в дереве.
        Возвращет будевские значения.
        '''
        
        return self.search(value) != None
    
    def get(self, value, metric=lambda x, y: np.abs(x - y)):
        '''
        Возвращает ближайщее значение к данному по метрике.
        Если значение есть в дереве, всегда возвращается нулевое значение метрики.
        В случае отсутствия значения, возращается разница между значением и его
        предполагаемым родительским узлом.
        '''
    
        if self.root is None:
            return None
    
        node, parent = self.__search_value(self.root, None, value)
        
        if not (node is None):
            return value, 0
        
        return parent.value, metric(parent.value, value)
        
        

### 3. Тестирование работы дерева на числовых значениях

In [4]:
t = BinTree()
t.insert(10)
t.insert(9)
t.insert(11)
t + 12
t + 5
len(t)

5

In [5]:
it = iter(t)
list(it)

[5, 9, 10, 11, 12]

In [6]:
t.get(13)

(12, 1)

In [7]:
t.contains(20)

False

In [8]:
t.delete(10)
list(iter(t))

[5, 9, 11, 12]

In [9]:
t - 5
list(iter(t))

[9, 11, 12]

In [10]:
t - 11
list(iter(t))

[9, 12]

In [11]:
t = BinTree()
for i in range(1000):
    t + i
len(t)

1000

### 4. Тестирование дерева на строках

В качестве метрики будем использовать расстояние Левенштейна
$$d(S_1, S_2) = D(M, N),$$
где

$$D(i, j) = 
\begin{cases}
    0, & i = 0, j = 0 \\
    i * c_2, & j = 0, i > 0 \\
    j * c_1, & i = 0, j > 0 \\
    \min(D(i - 1, j) + c_1, D(i, j - 1) + c_2, D(i - 1, j - 1) + c_3), & i > 0, j > 0 \\
 \end{cases}$$
 
 $c_1$, $c_2$ и $c_3$ - стоимость вставки, удаления и замены одного символа
 соответственно.

In [12]:
import numpy as np

def levenstein_dist(s1, s2, insert_cost=1, delete_cost=1, replace_cost=1):
    m = len(s1)
    n = len(s2)
    d = np.zeros((m + 1, n + 1), dtype=np.int)
    
    for j in range(1, n + 1):
        d[0][j] = d[0][j - 1] + insert_cost
    
    for i in range(1, m + 1):
        d[i][0] = d[i - 1][0] + delete_cost
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if s1[i - 1] != s2[j - 1]:
                d[i][j] = min(d[i - 1][j] + delete_cost,
                              d[i][j - 1] + insert_cost,
                              d[i - 1][j - 1] + replace_cost)
            else:
                d[i][j] = d[i - 1][j - 1]
    
    return d[m][n]

Генератор строк возьмем из предыдущего ДЗ:

In [13]:
import random
import string

def generate_random_str_list(min_len, max_len, count=1000, char_set=string.ascii_letters):
    if min_len >= max_len:
        raise ValueError("bad min/max values")
    
    return [''.join(random.choice(char_set) for i in range(min_len, random.randint(min_len + 1, max_len))) \
            for i in range(count)]

In [14]:
rand_strs = generate_random_str_list(8, 64, count=1000)
test_rand_strs = generate_random_str_list(8, 64, count=100)

In [15]:
t = BinTree()
for s in rand_strs:
    t + s
# Среди сгенерированных строк могут быть одинаковые.
len(t)

996

In [16]:
t.get(rand_strs[500], metric=levenstein_dist)

('EcHCmdALxf', 0)

In [17]:
test_rand_strs[50]

'jkofIKyFWZFUCqd'

In [18]:
t.get(test_rand_strs[50], metric=levenstein_dist)

('jhxahSbtWm', 13)